In [6]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)


def add_message(input, output):
    memory.save_context(
        {"input": input},
        {"output": output},
    )


add_message(1, 1)

In [7]:
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)
add_message(5, 5)

In [8]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [9]:
add_message(6, 6)
memory.load_memory_variables({})

{'history': [HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5'),
  HumanMessage(content='6'),
  AIMessage(content='6')]}